In [1]:
from glob import glob
import os
import re

from Bio import Entrez, GenBank
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
# TODO move the RefSeq uniprot mappings here too

In [2]:
# Inputs
mapping_folder = "../../data/mappings/"
raw_refseq_gpff_folder = mapping_folder+"/raw_refseq_mapping_gpff/" #"/n/groups/marks/databases/ukbiobank/users/rose/data/mapping/refseq_mapping_gff_updated/"

# Outputs
mapping_file_out = mapping_folder+"refseq_mapping_gff.tsv"

In [7]:
from datetime import date
def today():
    return date.today().strftime("%Y_%m_%d")
today()

'2023_05_22'

**If the files haven't been downloaded, download as follows:**

In [14]:
if len(glob(raw_refseq_gpff_folder+"/*.gpff*")) == 0:
    FTP_REFSEQ = "https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/mRNA_Prot/"
    # Download all the protein.gpff.gz files from the FTP server
    for i in range(1,12+1):
        filename = f"human.{i}.protein.gpff.gz"
        !wget {FTP_REFSEQ}/{filename} -O {raw_refseq_gpff_folder}/{today()}_{filename}
        # I like seeing the output so gzip separately
        !gzip -d {raw_refseq_gpff_folder}/{today()}_{filename}

--2023-05-22 05:48:44--  https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/mRNA_Prot//human.1.protein.gpff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 165.112.9.229, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1869331 (1.8M) [application/x-gzip]
Saving to: ‘../../data/mappings//raw_refseq_mapping_gpff//2023_05_22_human.1.protein.gpff.gz’

100%[======================================>] 1,869,331   --.-K/s   in 0.1s    

2023-05-22 05:48:44 (16.4 MB/s) - ‘../../data/mappings//raw_refseq_mapping_gpff//2023_05_22_human.1.protein.gpff.gz’ saved [1869331/1869331]

--2023-05-22 05:48:45--  https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/mRNA_Prot//human.2.protein.gpff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 165.112.9.228, 2607:f220:41f:250::229, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|

In [16]:
# Read and concat files into table
record_info = []

assert os.path.isdir(raw_refseq_gpff_folder)
file_list = glob(raw_refseq_gpff_folder+"/*.gpff")
assert len(file_list) > 0
for filepath in tqdm(file_list, desc="# of files processed"):
    records = []
    with open(filepath) as handle:
        # TMP(Lood): Cancel the NP/XP filter below.
        # if record.version.startswith("NP")   
        # Only keep the protein RefSeq accession prefixes starting with NP_ (not the more general XP_) 
        # https://www.ncbi.nlm.nih.gov/books/NBK50679/#RefSeqFAQ.what_is_the_difference_between
        
        records = [record for record in tqdm(GenBank.parse(handle), leave=False)]
    
    for record in records:
        info = {'protein':record.version,'length':record.size,'definition':record.definition}
        cds = [i for i in record.features if i.key=='CDS'][0]
        for qualifier in cds.qualifiers:
            key = re.sub('/|=','',qualifier.key)
            value = re.sub('"','',qualifier.value)
            if key == 'db_xref':
                info[value.split(':')[0]] = value.split(':')[-1]
            else:
                info[key] = value
        #info.extend([re.sub('"','',i.value) for i in cds.qualifiers])
        record_info.append(info)

# of files processed:   0%|          | 0/12 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [44]:
# Concat into df, store set of all transcripts in clinvar e.g.? Or do within ClinVar?
df_mapping = pd.DataFrame(record_info)
df_mapping.head()

,protein,length,definition,gene,coded_by,GeneID,gene_synonym,CCDS,HGNC,note,MIM,GO_function,IMGT/GENE-DB,exception,ribosomal_slippage,GO_component,GO_process
0,NP_001355183.1,382,killer cell immunoglobulin-like receptor 3DS1-...,LOC112268355,NM_001368254.1:47..1195,112268355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NP_001337906.1,44,putative keratin-associated protein 20-4 [Homo...,KRTAP20-4,NM_001350977.1:32..166,100151643,KAP20.4,CCDS86982.1,34002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NP_001243796.1,530,ubiquitin specific peptidase 17 like family me...,USP17L30,NM_001256867.1:1..1593,728419,NaN,CCDS59471.1,44458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NP_001229257.1,530,ubiquitin specific peptidase 17 like family me...,USP17L26,NM_001242328.1:1..1593,728379,NaN,CCDS59466.1,44454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NP_001243802.1,530,ubiquitin carboxyl-terminal hydrolase 17-like ...,USP17L1,NM_001256873.1:1..1593,401447,USP17L1P,CCDS78298.1,37182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
len(df_mapping)

66931

In [48]:
# Skip the variants with ribosomal slippage (10 out of 67k), which will have "join()" in the coded_by field
display(df_mapping[(df_mapping["coded_by"].str.count(":") != 1)].head(1))
assert (df_mapping["coded_by"].str.count(":") != 1).equals(df_mapping["ribosomal_slippage"].notna())

# Split up the AA annotation
df_mapping_clean = df_mapping.loc[df_mapping["ribosomal_slippage"].isna()].copy()
assert (df_mapping_clean["coded_by"].str.count(":") == 1).all()

df_mapping_clean[["mrna", "coding_pos"]] = df_mapping_clean["coded_by"].str.split(":", expand=True)

,protein,length,definition,gene,coded_by,GeneID,gene_synonym,CCDS,HGNC,note,MIM,GO_function,IMGT/GENE-DB,exception,ribosomal_slippage,GO_component,GO_process
20642,NP_055883.2,708,retrotransposon-derived protein PEG10 isoform ...,PEG10,"join(NM_015068.3:480..1436, NM_015068.3:1436.....",23089,EDR; HB-1; Mar2; Mart2; MEF3L; RGAG3; RTL2; SIRH1,NaN,14005,protein translation is dependent on -1 ribosom...,609810,NaN,NaN,NaN,,NaN,NaN


In [55]:
# Write out mapping file
if os.path.isfile(mapping_file_out):
    print("Warning: Overwriting existing mapping file.")
df_mapping_clean.to_csv(mapping_file_out, sep="\t", index=False)

In [ ]:
# Missing mappings (TODO test this)
with open('/n/groups/marks/databases/ukbiobank/users/rose/data/mapping/refseq_mapping_gff_updated/missing.gpff', 'w') as file:
    for protein in all_transcripts - set(df.protein):
        handle = Entrez.efetch(db='protein', 
                               id=protein, 
                               rettype='gb',
                               retmode='txt')
        gb = handle.read()
        handle.close()
        file.write(gb)
        file.write('\n')